In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff)
#nbTuring.turnprogress(false);

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

Turing model

In [3]:
@model m12_6(total_tools, log_pop, society) = begin
    # Total num of y
    N = length(total_tools)
    # priors
    α ~ Normal(0, 10)
    βp ~ Normal(0, 1)
    # Separate σ priors for each society
    σ_society ~ Truncated(Cauchy(0, 1), 0, Inf)
    # Number of unique societies in the data set
    N_society = length(unique(society)) #10
    # Vector of societies (1,..,10) which we'll set priors on
    α_society = Vector{Real}(undef, N_society)
    # For each society [1,..,10] set a prior N(0, σ_society)
    α_society ~ [Normal(0, σ_society)]

    for i ∈ 1:N
        λ = exp(α + α_society[society[i]] + βp*log_pop[i])
        total_tools[i] ~ Poisson(λ)
    end
end

m12_6 (generic function with 4 methods)

Sample

In [4]:
posterior = sample(m12_6(d[:total_tools], d[:log_pop],
    d[:society]), Turing.NUTS(4000, 1000, 0.95));
# Fix the inclusion of adaptation samples
posterior2 = MCMCChain.Chains(posterior.value[1001:4000,:,:], names=posterior.names)

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, -2.6887e6, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, -2.80234e6, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, -1.35465e5, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been fou

Object of type "Chains{Float64}"

Iterations = 1:3000
Thinning interval = 1
Chains = 1
Samples per chain = 3000

Union{Missing, Float64}[0.511546 0.079982 … 0.218663 0.00526838; 0.306171 0.0664622 … 0.129788 0.00526838; … ; 1.28427 0.313298 … 0.129042 0.00526838; 1.30485 0.531285 … 0.106524 0.00526838]

Results rethinking

In [5]:
m126rethinking = "
              Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
a              1.11   0.75      -0.05       2.24  1256    1
bp             0.26   0.08       0.13       0.38  1276    1
a_society[1]  -0.20   0.24      -0.57       0.16  2389    1
a_society[2]   0.04   0.21      -0.29       0.38  2220    1
a_society[3]  -0.05   0.19      -0.36       0.25  3018    1
a_society[4]   0.32   0.18       0.01       0.60  2153    1
a_society[5]   0.04   0.18      -0.22       0.33  3196    1
a_society[6]  -0.32   0.21      -0.62       0.02  2574    1
a_society[7]   0.14   0.17      -0.13       0.40  2751    1
a_society[8]  -0.18   0.19      -0.46       0.12  2952    1
a_society[9]   0.27   0.17      -0.02       0.52  2540    1
a_society[10] -0.10   0.30      -0.52       0.37  1433    1
sigma_society  0.31   0.13       0.11       0.47  1345    1
";

Describe the posterior samples

In [6]:
describe(posterior2)

Iterations = 1:3000
Thinning interval = 1
Chains = 1
Samples per chain = 3000

Empirical Posterior Estimates:
                   Mean           SD        Naive SE        MCSE         ESS    
            α   0.9789482151 0.707105495 0.01290992101 0.09145045945   59.785607
 α_society[9]   0.2802624743 0.178611963 0.00326099337 0.01221015353  213.982786
       lf_num   0.0000000000 0.000000000 0.00000000000 0.00000000000         NaN
 α_society[8]  -0.1656160241 0.185150623 0.00338037242 0.00943854300  384.804761
      elapsed   0.0583991339 0.006626841 0.00012098901 0.00010653978 3000.000000
      epsilon   0.0052683765 0.000000000 0.00000000000 0.00000000000         NaN
 α_society[3]  -0.0205704670 0.191450604 0.00349539381 0.01049993370  332.460740
 α_society[1]  -0.1745270821 0.226456252 0.00413450658 0.01409274896  258.212462
α_society[10]  -0.1205820633 0.290395142 0.00530186566 0.03191225118   82.806382
 α_society[5]   0.0577537810 0.174798482 0.00319136904 0.00978545836  319.089832

End of m12.6t.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*